In [1]:
import pandas as pd
from tqdm import tqdm
import obspy
import numpy as np
from obspy.core.utcdatetime import UTCDateTime
import sys
import h5py 
sys.path.append("/home/niyiyu/Research/pnwstore/")

import matplotlib.pyplot as plt
from obspy.core.utcdatetime import UTCDateTime
from pnwstore.catalog import QuakeClient
from pnwstore.station import StationClient

from collections import Counter

In [2]:
quakeclient = QuakeClient("cascadia", "pnwstore")
stationclient = StationClient("cascadia", "pnwstore")
pd.set_option('display.max_columns', None)

## metadata to be added
- station location:
       latitude, longitude, elevation
- trace_P/S_polarity
    - added in mpi_extract_pnwml.py
- trace_[P/S]_onset
    - added in mpi_extract_pnwml.py
    
- trace_SNR
- trace_uncertainty

- source_magnitude_type
    - added in mpi_extract_pnwml.py
- source_magnitude_uncertainty
    - added in mpi_extract_pnwml.py
- source_horizontal_uncertainty
    - added in mpi_extract_pnwml.py

In [3]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")
# df['year'] = df.apply(lambda row: int(row['trace_start_time'].split("-")[0]), axis = 1)
df

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_type,source_depth_km,source_magnitude,source_magnitude_type,source_magnitude_uncertainty,source_depth_uncertainty_km,source_horizontal_uncertainty_km,station_network_code,trace_channel,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,trace_S_arrival_sample,trace_P_arrival_sample,trace_S_arrival_uncertainty_s,trace_P_arrival_uncertainty_s,trace_P_polarity,trace_S_onset,trace_P_onset,trace_snr_db
0,uw61899186,2022-12-19T16:28:24.960000Z,47.703,-122.632,earthquake,24.96,1.87,ml,0.236,0.56,0.34,UW,EN,KITP,NaN,TBA,TBA,TBA,"bucket1$0,:3,:15001",100,2022-12-19T16:27:34.960000Z,5805,5405,0.06,0.06,positive,impulsive,impulsive,NaN
1,uw61899186,2022-12-19T16:28:24.960000Z,47.703,-122.632,earthquake,24.96,1.87,ml,0.236,0.56,0.34,UW,EN,SWID,NaN,TBA,TBA,TBA,"bucket1$1,:3,:15001",100,2022-12-19T16:27:34.960000Z,6369,5738,0.06,0.15,undecidable,impulsive,emergent,NaN
2,uw61502338,2022-12-19T13:26:44.310000Z,48.483,-121.707,earthquake,9.08,0.68,md,0.124,0.94,0.70,UW,EN,PLBD,NaN,TBA,TBA,TBA,"bucket1$2,:3,:15001",100,2022-12-19T13:25:54.310000Z,5366,5207,0.15,0.03,negative,emergent,impulsive,NaN
3,uw61899161,2022-12-19T13:26:12.190000Z,48.482,-121.709,earthquake,8.37,1.34,ml,0.106,0.52,0.29,UW,EN,PUBD,NaN,TBA,TBA,TBA,"bucket1$3,:3,:15001",100,2022-12-19T13:25:22.190000Z,5628,5358,0.06,0.06,negative,impulsive,impulsive,NaN
4,uw61899086,2022-12-18T22:04:56.950000Z,48.411,-121.849,earthquake,5.59,1.38,ml,0.096,3.56,0.46,UW,EN,PLBD,NaN,TBA,TBA,TBA,"bucket1$4,:3,:15001",100,2022-12-18T22:04:06.950000Z,5581,5342,0.15,0.30,undecidable,emergent,emergent,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,uw61822386,2022-03-14T19:49:15.010000Z,48.895,-121.990,earthquake,-0.67,0.84,md,0.264,1.30,0.62,UW,EN,MBKE,NaN,TBA,TBA,TBA,"bucket1$362,:3,:15001",100,2022-03-14T19:48:25.010000Z,5378,5196,0.06,0.06,undecidable,impulsive,impulsive,NaN
363,uw61822721,2022-03-16T14:40:32.220000Z,48.751,-121.836,earthquake,30.96,0.95,md,0.176,1.36,1.05,UW,EN,PLBD,NaN,TBA,TBA,TBA,"bucket1$363,:3,:15001",100,2022-03-16T14:39:42.220000Z,6085,5642,0.06,0.06,undecidable,impulsive,impulsive,NaN
364,uw61822716,2022-03-16T10:56:52.430000Z,47.308,-122.020,earthquake,21.96,1.33,ml,0.089,0.66,0.40,UW,EN,ELW,NaN,TBA,TBA,TBA,"bucket1$364,:3,:15001",100,2022-03-16T10:56:02.430000Z,5922,5507,0.06,0.06,negative,impulsive,impulsive,NaN
365,uw61803812,2022-01-25T12:37:50.480000Z,47.961,-121.969,earthquake,13.28,1.19,ml,0.119,0.60,0.30,UW,EN,MBPA,NaN,TBA,TBA,TBA,"bucket1$365,:3,:15001",100,2022-01-25T12:37:00.480000Z,5491,5277,0.06,0.06,undecidable,impulsive,impulsive,NaN


In [10]:
df = df.drop(index = df[df['year'] <=2005].index, axis = 0)
df = df.reset_index(drop = True)
df.to_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv", sep = ',', index=False)

## check weird arrival times

In [4]:
# df = pd.read_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv")
for idx, i in df.iterrows():
    if i['trace_S_arrival_sample'] < 0 or i['trace_P_arrival_sample'] >15000 or i['trace_P_arrival_sample'] < 0 or i['trace_S_arrival_sample'] >15000:
        print(i['source_id'], i['trace_P_arrival_sample'])
        df = df.drop(axis = 0, index = idx)
df = df.reset_index(drop = True)
# df.to_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv", sep = ',', index=False)

In [7]:
df = pd.read_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv")
for idx, i in df.iterrows():
    if i['trace_S_arrival_sample'] < i['trace_P_arrival_sample']:
        print(i['source_id'], i['trace_P_arrival_sample'])
        df = df.drop(axis = 0, index = idx)
df = df.reset_index(drop = True)
# df.to_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv", sep = ',', index=False)

## calculate SNR

In [4]:
def cal_snr(data, percentile = 98, string = False):
    snrs = []
    for icha in range(3):
        snrs.append(20 * np.log(np.percentile(np.abs(data[icha, ids-100:ids + 200]), percentile)/
              np.percentile(np.abs(data[icha, idp-800:idp]), percentile))/np.log(10))
    if string:
        return "%.3f|%.3f|%.3f" % (snrs[0], snrs[1], snrs[2])
    else:
        return snrs

In [5]:
# df = pd.read_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv")

In [6]:
f = h5py.File(f"../data/extract/proc2022_waveforms.hdf5", "r")
snrs = []
for idx in tqdm(range(len(df))):
    ids = df.iloc[idx]['trace_S_arrival_sample']
    idp = df.iloc[idx]['trace_P_arrival_sample']
    tn = df.iloc[idx]['trace_name']
    bucket, narray = tn.split('$')
    x, y, z = iter([int(i) for i in narray.split(',:')])
    data = f['/data/%s' % bucket][x, :y, :z]
    df.iloc[idx, 28] = cal_snr(data, string = True)
f.close()

100%|██████████| 367/367 [00:00<00:00, 785.07it/s]


In [7]:
df

,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_type,source_depth_km,source_magnitude,source_magnitude_type,source_magnitude_uncertainty,source_depth_uncertainty_km,source_horizontal_uncertainty_km,station_network_code,trace_channel,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,trace_S_arrival_sample,trace_P_arrival_sample,trace_S_arrival_uncertainty_s,trace_P_arrival_uncertainty_s,trace_P_polarity,trace_S_onset,trace_P_onset,trace_snr_db
0,uw61899186,2022-12-19T16:28:24.960000Z,47.703,-122.632,earthquake,24.96,1.87,ml,0.236,0.56,0.34,UW,EN,KITP,NaN,TBA,TBA,TBA,"bucket1$0,:3,:15001",100,2022-12-19T16:27:34.960000Z,5805,5405,0.06,0.06,positive,impulsive,impulsive,1.888|3.370|2.793
1,uw61899186,2022-12-19T16:28:24.960000Z,47.703,-122.632,earthquake,24.96,1.87,ml,0.236,0.56,0.34,UW,EN,SWID,NaN,TBA,TBA,TBA,"bucket1$1,:3,:15001",100,2022-12-19T16:27:34.960000Z,6369,5738,0.06,0.15,undecidable,impulsive,emergent,8.425|2.801|1.200
2,uw61502338,2022-12-19T13:26:44.310000Z,48.483,-121.707,earthquake,9.08,0.68,md,0.124,0.94,0.70,UW,EN,PLBD,NaN,TBA,TBA,TBA,"bucket1$2,:3,:15001",100,2022-12-19T13:25:54.310000Z,5366,5207,0.15,0.03,negative,emergent,impulsive,17.274|12.727|15.213
3,uw61899161,2022-12-19T13:26:12.190000Z,48.482,-121.709,earthquake,8.37,1.34,ml,0.106,0.52,0.29,UW,EN,PUBD,NaN,TBA,TBA,TBA,"bucket1$3,:3,:15001",100,2022-12-19T13:25:22.190000Z,5628,5358,0.06,0.06,negative,impulsive,impulsive,11.211|10.816|13.325
4,uw61899086,2022-12-18T22:04:56.950000Z,48.411,-121.849,earthquake,5.59,1.38,ml,0.096,3.56,0.46,UW,EN,PLBD,NaN,TBA,TBA,TBA,"bucket1$4,:3,:15001",100,2022-12-18T22:04:06.950000Z,5581,5342,0.15,0.30,undecidable,emergent,emergent,17.426|12.707|10.227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,uw61822386,2022-03-14T19:49:15.010000Z,48.895,-121.990,earthquake,-0.67,0.84,md,0.264,1.30,0.62,UW,EN,MBKE,NaN,TBA,TBA,TBA,"bucket1$362,:3,:15001",100,2022-03-14T19:48:25.010000Z,5378,5196,0.06,0.06,undecidable,impulsive,impulsive,14.069|13.906|1.198
363,uw61822721,2022-03-16T14:40:32.220000Z,48.751,-121.836,earthquake,30.96,0.95,md,0.176,1.36,1.05,UW,EN,PLBD,NaN,TBA,TBA,TBA,"bucket1$363,:3,:15001",100,2022-03-16T14:39:42.220000Z,6085,5642,0.06,0.06,undecidable,impulsive,impulsive,7.252|5.839|0.722
364,uw61822716,2022-03-16T10:56:52.430000Z,47.308,-122.020,earthquake,21.96,1.33,ml,0.089,0.66,0.40,UW,EN,ELW,NaN,TBA,TBA,TBA,"bucket1$364,:3,:15001",100,2022-03-16T10:56:02.430000Z,5922,5507,0.06,0.06,negative,impulsive,impulsive,6.408|9.036|3.342
365,uw61803812,2022-01-25T12:37:50.480000Z,47.961,-121.969,earthquake,13.28,1.19,ml,0.119,0.60,0.30,UW,EN,MBPA,NaN,TBA,TBA,TBA,"bucket1$365,:3,:15001",100,2022-01-25T12:37:00.480000Z,5491,5277,0.06,0.06,undecidable,impulsive,impulsive,11.123|10.574|1.193


In [11]:
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

## filter out low SNR

In [8]:
# df = pd.read_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv")
print(len(df))
for idx, i in df.iterrows():
    snrs = [float(s) for s in i['trace_snr_db'].split('|')]
    for s in snrs:
        if s < -20 or s > 80:
            print("drop!")
            df = df.drop(axis = 0, index = idx)
            break
df = df.reset_index(drop = True)
print(len(df))
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

367
drop!
366


## EPH2 EH -> HH

In [29]:
df = pd.read_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv")
df.iloc[df[(df['station_code'] == 'EPH2') & (df['trace_channel'] == 'EH')].index, 12] = "HH"
df.to_csv("/home/niyiyu/.seisbench/datasets/pnw/metadata.csv", sep = ',', index=False)

## UW.GNW.BH

## check location 

In [9]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")
df['station_location_code'].unique()

array([ nan])

In [10]:
df.iloc[df[pd.isna(df['station_location_code'])].index, 14] = '--'
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

## add lat lon elevation

In [11]:
df = pd.read_csv("../data/extract/proc2022_metadata.csv")

In [12]:
error = []
for idx, i in tqdm(df.iterrows(), total = len(df)):
    if df.iloc[idx, 15] == "TBA":
        network = df.iloc[idx, 11]
        station = df.iloc[idx, 13]
        channel = df.iloc[idx, 12] + "Z"
        time = UTCDateTime(df.iloc[idx, 1])
        result = stationclient.query(network = network, station = station, channel = channel, time = time)
        
        try:
            assert len(result) >= 1
            df.iloc[idx, 15] = result.iloc[0, 5]         # latitude
            df.iloc[idx, 16] = result.iloc[0, 6]         # longitude
            df.iloc[idx, 17] = result.iloc[0, 7]         # elevation
            
        except:
            print(idx)
            error.append([idx, i])

100%|██████████| 366/366 [00:16<00:00, 22.38it/s]


In [13]:
df.to_csv("../data/extract/proc2022_metadata.csv", sep = ',', index=False)

## merge quarry blast/others into explosion/earthquake

In [14]:
df = pd.read_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv")

In [18]:
df.iloc[df[df['source_type'] == 'quarry blast'].index, 4] = 'explosion'
df.iloc[df[df['source_id'] == 'uw61553011'].index, 4] = 'earthquake'
df.iloc[df[df['source_id'] == 'uw61733307'].index, 4] = 'earthquake'
df.iloc[df[df['source_id'] == 'uw61742182'].index, 4] = 'explosion'

In [19]:
df.iloc[df[df['source_id'] == 'uw61788642'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61587947'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61585656'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61565312'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61166917'].index, 4] = 'surface event'
df.iloc[df[df['source_id'] == 'uw61212391'].index, 4] = 'surface event'

In [20]:
df.iloc[df[df['source_id'] == 'uw61542931'].index, 4] = 'thunder'

In [21]:
df.iloc[df[df['source_id'] == 'uw61500398'].index, 4] = 'low-frequency earthquake'

In [14]:
Counter(df['source_type'])

Counter({'earthquake': 348, 'explosion': 18})

In [30]:
df.to_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv", sep = ',', index=False)

## remove event of sonic bloom & low-frequency earthquake

In [31]:
df = pd.read_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv")

In [32]:
df.drop(axis = 1, index = df[df['source_type']=='sonic boom'].index, inplace = True)
df.drop(axis = 1, index = df[df['source_type']=='low-frequency earthquake'].index, inplace = True)
df.drop(axis = 1, index = df[df['source_type']=='surface event'].index, inplace = True)
df.drop(axis = 1, index = df[df['source_type']=='thunder'].index, inplace = True)

In [33]:
df = df.reset_index(drop = True)

In [34]:
df.to_csv("/home/niyiyu/Research/PNW-ML/product/accelerometer_metadata.csv", sep = ',', index=False)

In [35]:
Counter(df['station_network_code'])

Counter({'UW': 5729, 'TA': 53, 'UO': 13, 'CN': 7})